## Missing Value Imputation ,Word Segmentation , StopWords Removing
### (Using Testing data for example, procedure here also applies to training data)

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
from matplotlib.font_manager import _rebuild

_rebuild() #reload一下
plt.rcParams['font.sans-serif']=['SimHei']

In [2]:
df1 = pd.read_excel("test1.xlsx", encoding="UTF-8")

In [3]:
df1.head()

,id,tid1,tid2,title1_zh,title2_zh,title1_en,title2_en
0,321187,167562,59521,萨拉赫人气爆棚!埃及总统大选未参选获百万选票 现任总统压力山大,辟谣！里昂官方否认费基尔加盟利物浦，难道是价格没谈拢？,egypt 's presidential election failed to win m...,Lyon! Lyon officials have denied that Felipe F...
1,321190,167564,91315,萨达姆被捕后告诫美国的一句话，发人深思,10大最让美国人相信的荒诞谣言，如蜥蜴人掌控着美国,A message from Saddam Hussein after he was cap...,The Top 10 Americans believe that the Lizard M...
2,321189,167563,167564,萨达姆此项计划没有此国破坏的话，美国还会对伊拉克发动战争吗,萨达姆被捕后告诫美国的一句话，发人深思,Will the United States wage war on Iraq withou...,A message from Saddam Hussein after he was cap...
3,321193,167564,160994,萨达姆被捕后告诫美国的一句话，发人深思,被绞刑处死的萨达姆是替身？他的此男人举动击破替身谣言！,A message from Saddam Hussein after he was cap...,The hanging Saddam is a surrogate? This man's ...
4,321191,167564,15084,萨达姆被捕后告诫美国的一句话，发人深思,中国川贝枇杷膏在美国受到热捧？纯属谣言！,A message from Saddam Hussein after he was cap...,Chinese loquat loquat plaster in America? Pure...


In [4]:
df1.shape

(80126, 7)

In [6]:
cols = ['title1_zh','title2_zh']
df1 = df1.loc[:, cols]

,title1_zh,title2_zh
0,萨拉赫人气爆棚!埃及总统大选未参选获百万选票 现任总统压力山大,辟谣！里昂官方否认费基尔加盟利物浦，难道是价格没谈拢？
1,萨达姆被捕后告诫美国的一句话，发人深思,10大最让美国人相信的荒诞谣言，如蜥蜴人掌控着美国
2,萨达姆此项计划没有此国破坏的话，美国还会对伊拉克发动战争吗,萨达姆被捕后告诫美国的一句话，发人深思


In [7]:
df1.isna().any()

title1_zh    False
title2_zh     True
dtype: bool

In [8]:
df1.title2_zh.fillna('UNKNOWN', inplace=True)
df1.isna().any()

title1_zh    False
title2_zh    False
dtype: bool

In [9]:
import thulac
t=thulac.thulac()

Model loaded succeed


In [10]:
def thulac_tokenizer(text):
    words = t.cut(text)
    return ' '.join([word[0] for word in words if word[1] != 'w'])

In [11]:
t.cut('UNKNOWN')

[['UNKNOWN', 'x']]

In [12]:
thulac_tokenizer('UNKNOWN')

'UNKNOWN'

In [13]:
def process(data):
    res = data.apply(thulac_tokenizer)
    return res

def check_merge_idx(data, res):
    assert((data.index == res.index).all(), 'Something error when merge data')

<ipython-input-13-407635f9fd0a>:6: SyntaxWarning: assertion is always true, perhaps remove parentheses?
  assert((data.index == res.index).all(), 'Something error when merge data')


In [16]:
import numbers
for i,string in enumerate(df1.loc[:, 'title2_zh']):
    if isinstance(string,list):
        print(i,string)

In [18]:
np.all(df1.index == df1.title1_zh.index)

True

In [19]:
import fool

C:\Users\user\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [ ]:
#tokenize data
df1['title2_tokenized'] = process(df1.loc[:, 'title2_zh'])
df1['title1_tokenized'] = process(df1.loc[:, 'title1_zh'])

In [23]:
#name entity recognition 
df1['title1_ner']=df1['title1_zh'].apply(fool.ner)
df1['title2_ner']=df1['title2_zh'].apply(fool.ner)

In [25]:
#turn string to list
df1['title1_tokenized']=df1['title1_tokenized'].apply(lambda x: x.split(' '))
df1['title2_tokenized']=df1['title2_tokenized'].apply(lambda x: x.split(' '))

In [26]:
df1.head()

,title1_zh,title2_zh,title2_tokenized,title1_tokenized,title1_ner,title2_ner
0,萨拉赫人气爆棚!埃及总统大选未参选获百万选票 现任总统压力山大,辟谣！里昂官方否认费基尔加盟利物浦，难道是价格没谈拢？,"[辟谣, 里昂, 官方, 否, 认费, 基尔, 加盟, 利物浦, 难道, 是, 价, 格没,...","[萨拉赫, 人气, 爆棚, 埃及, 总统, 大选, 未, 参选, 获, 百万, 选票, , ...","[[(0, 4, person, 萨拉赫), (10, 13, job, 总统), (25,...","[[(9, 18, location, 费基尔加盟利物浦)]]"
1,萨达姆被捕后告诫美国的一句话，发人深思,10大最让美国人相信的荒诞谣言，如蜥蜴人掌控着美国,"[10, 大, 最, 让, 美国, 人, 相信, 的, 荒诞, 谣言, 如, 蜥蜴人, 掌控...","[萨达姆, 被捕, 后, 告诫, 美国, 的, 一, 句, 话, 发人深思]","[[(0, 4, person, 萨达姆), (8, 11, location, 美国)]]","[[(5, 8, location, 美国), (23, 26, location, 美国)]]"
2,萨达姆此项计划没有此国破坏的话，美国还会对伊拉克发动战争吗,萨达姆被捕后告诫美国的一句话，发人深思,"[萨达姆, 被捕, 后, 告诫, 美国, 的, 一, 句, 话, 发人深思]","[萨达姆, 此, 项, 计划, 没有, 此国, 破坏, 的, 话, 美国, 还, 会, 对,...","[[(0, 4, person, 萨达姆), (16, 19, location, 美国),...","[[(0, 4, person, 萨达姆), (8, 11, location, 美国)]]"
3,萨达姆被捕后告诫美国的一句话，发人深思,被绞刑处死的萨达姆是替身？他的此男人举动击破替身谣言！,"[被, 绞刑, 处死, 的, 萨达姆, 是, 替身, 他, 的, 此, 男人, 举动, 击破...","[萨达姆, 被捕, 后, 告诫, 美国, 的, 一, 句, 话, 发人深思]","[[(0, 4, person, 萨达姆), (8, 11, location, 美国)]]","[[(6, 10, person, 萨达姆)]]"
4,萨达姆被捕后告诫美国的一句话，发人深思,中国川贝枇杷膏在美国受到热捧？纯属谣言！,"[中国, 川贝, 枇杷膏, 在, 美国, 受到, 热捧, 纯属, 谣言]","[萨达姆, 被捕, 后, 告诫, 美国, 的, 一, 句, 话, 发人深思]","[[(0, 4, person, 萨达姆), (8, 11, location, 美国)]]","[[(0, 3, location, 中国), (8, 11, location, 美国)]]"


In [27]:
#You should download Chinese Stopwords first and save it in the location as your codes
stopWords=[]
with open('chnstopwords.txt', 'r', encoding='UTF-8') as file:
    for data in file.readlines():
        data = data.strip()
        stopWords.append(data)

In [28]:
len(stopWords)

746

In [29]:
def filterstop(sentence):
    ls=[]
    for word in sentence:
        if word not in stopWords:
            ls.append(word)
    return ls

In [30]:
#Remove Stopwords
df1['title1_tokenized']=df1['title1_tokenized'].apply(filterstop)
df1['title2_tokenized']=df1['title2_tokenized'].apply(filterstop)

In [31]:
df1.head()

,title1_zh,title2_zh,title2_tokenized,title1_tokenized,title1_ner,title2_ner
0,萨拉赫人气爆棚!埃及总统大选未参选获百万选票 现任总统压力山大,辟谣！里昂官方否认费基尔加盟利物浦，难道是价格没谈拢？,"[辟谣, 里昂, 官方, 否, 认费, 基尔, 加盟, 利物浦, 难道, 价, 格没, 谈拢]","[萨拉赫, 人气, 爆棚, 埃及, 总统, 大选, 未, 参选, 获, 百万, 选票, , ...","[[(0, 4, person, 萨拉赫), (10, 13, job, 总统), (25,...","[[(9, 18, location, 费基尔加盟利物浦)]]"
1,萨达姆被捕后告诫美国的一句话，发人深思,10大最让美国人相信的荒诞谣言，如蜥蜴人掌控着美国,"[10, 美国, 相信, 荒诞, 谣言, 蜥蜴人, 掌控, 美国]","[萨达姆, 被捕, 告诫, 美国, 句, 话, 发人深思]","[[(0, 4, person, 萨达姆), (8, 11, location, 美国)]]","[[(5, 8, location, 美国), (23, 26, location, 美国)]]"
2,萨达姆此项计划没有此国破坏的话，美国还会对伊拉克发动战争吗,萨达姆被捕后告诫美国的一句话，发人深思,"[萨达姆, 被捕, 告诫, 美国, 句, 话, 发人深思]","[萨达姆, 项, 计划, 没有, 此国, 破坏, 话, 美国, 会, 伊拉克, 发动, 战争]","[[(0, 4, person, 萨达姆), (16, 19, location, 美国),...","[[(0, 4, person, 萨达姆), (8, 11, location, 美国)]]"
3,萨达姆被捕后告诫美国的一句话，发人深思,被绞刑处死的萨达姆是替身？他的此男人举动击破替身谣言！,"[绞刑, 处死, 萨达姆, 替身, 男人, 举动, 击破, 替身, 谣言]","[萨达姆, 被捕, 告诫, 美国, 句, 话, 发人深思]","[[(0, 4, person, 萨达姆), (8, 11, location, 美国)]]","[[(6, 10, person, 萨达姆)]]"
4,萨达姆被捕后告诫美国的一句话，发人深思,中国川贝枇杷膏在美国受到热捧？纯属谣言！,"[中国, 川贝, 枇杷膏, 美国, 热捧, 纯属, 谣言]","[萨达姆, 被捕, 告诫, 美国, 句, 话, 发人深思]","[[(0, 4, person, 萨达姆), (8, 11, location, 美国)]]","[[(0, 3, location, 中国), (8, 11, location, 美国)]]"


## Generate Useful Attributes

#### 1.Absolute Diffence of length of words in title1 and title2

In [70]:
df1['lenword']=abs(df1['title1_tokenized'].apply(len)-df1['title2_tokenized'].apply(len))

#### 2.Absolute Diffence of length of characters in title1 and title2

In [72]:
df1['lenchar']=abs(df1['title1_zh'].apply(len)-df1['title2_zh'].apply(len))

In [74]:
np.corrcoef(df1['lenchar'],df1['lenword'])

array([[1.       , 0.6721087],
       [0.6721087, 1.       ]])

#### 3.Number of words shared in title1 and title2 (repetition counts)

In [ ]:
def intersection(lst1, lst2): 
    lst3 = [value for value in lst1 if value in lst2] 
    return len(lst3)

In [76]:
df1['wordmatch']=df1.apply(lambda row:intersection(row['title1_tokenized'],row['title2_tokenized']),axis=1)

#### 4.Number of words in title2 but not in title 1 (repetition counts)

In [ ]:
def hypextrals(lst1, lst2): 
    lst3 = [value for value in lst2 if value not in lst1] 
    return len(lst3)

In [77]:
df1['hypextra']=df1.apply(lambda row:hypextrals(row['title1_tokenized'],row['title2_tokenized'),axis=1)

In [78]:
np.corrcoef(df1['wordmatch'],df1['hypextra'])

array([[ 1.        , -0.62793109],
       [-0.62793109,  1.        ]])

In [84]:
#Remove position information in name entity recognition
def nonumner(sent):
    s=[]
    for word in sent[0]:
        s.append(word[-2:])
    return s    

In [85]:
#test
nonumner(df1.loc[2,'title2_ner'])

[('person', '萨达姆'), ('location', '美国')]

In [86]:
df1['title1_nonumner']=df1['title1_ner'].apply(nonumner)
df1['title2_nonumner']=df1['title2_ner'].apply(nonumner)

#### 5.Number of NERs shared in title1 and title2 (repetition counts)

In [90]:
df1['nerwordmatch']=df1.apply(lambda row:intersection(row['title1_nonumner'],row['title2_nonumner']),axis=1)

In [102]:
df1['nerwordmatch'].value_counts()

0    54011
1    16223
2     7451
3     2039
4      369
5       32
6        1
Name: nerwordmatch, dtype: int64

#### 6.Number of NERs in title2 but not in title 1 (repetition counts)

In [92]:
df1['nerhypextra']=df1.apply(lambda row:hypextrals(row['title1_nonumner'],row['title2_nonumner']),axis=1)

In [94]:
df1['nerhypextra'].value_counts()

0     44021
1     24147
2      8825
3      2411
4       604
5        94
6        19
7         4
10        1
Name: nerhypextra, dtype: int64

In [95]:
np.corrcoef(df1['nerwordmatch'],df1['nerhypextra'])

array([[ 1.        , -0.11706049],
       [-0.11706049,  1.        ]])

In [ ]:
from wordcloud import WordCloud

In [7]:
#You need to download one of the Chinese fonts to your local drive first to display the chinese character in the plot,here I download Simhei Font
font = r'C:\Users\user\Anaconda3\pkgs\matplotlib-2.2.2-py36h153e9ff_1\Lib\site-packages\matplotlib\mpl-data\fonts\ttf\SimHei.ttf'

In [ ]:
train_qsdis =df1.loc[df1['label'] == 'disagreed','title2_zh'].astype(str)

In [ ]:
#Examine which word appears most frequently in title2
cloud = WordCloud(width=1440, height=1080, font_path=font).generate(" ".join(train_qsdis.astype(str)))
plt.figure(figsize=(20, 15))
plt.imshow(cloud)
plt.axis('off')
cloud.to_file('show_Chinese.png')

In [ ]:
#Choose seven key words appear most frequently in title2
def iscontentword(word):
     return word in ('辟谣','网传','谣言','假的','假消息','网警','警方')

In [ ]:
len(list(filter(iscontentword,['辟谣','网传','谣言','假的','假的','我'])))

#### 6.Number of key words appear in title2 (repetition counts)

In [ ]:
df1['iskey']=df1['title2_tokenized'].apply(lambda x:len(list(filter(iscontentword,x))))

In [ ]:
df1.iskey.value_counts()

In [ ]:
#Save the processed data to local drive for future analyzation
df1.to_csv('datatestsplattrs.csv')